# Simulation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import facets_drivers
import os
user = os.environ.get('USER')

num_cores = 16
theory_dump_directory = f"/data/id01/inhouse/{user}/scratch/ewoks_02_theory/"
simulation_dump_directory = f"/data/id01/inhouse/{user}/scratch/ewoks_03_simulation/"

In [3]:
from ewoks import execute_graph

nodes = [
    {
        "id": "get_orientation",
        "task_type": "class",
        "task_identifier": "facets_drivers.theory.GetOrientation",
    },
    {
        "id": "nanoresculpted",
        "task_type": "class",
        "task_identifier": "facets_drivers.simulation.RunNanoReSculpted",
    },
    {
        "id": "lammps",
        "task_type": "class",
        "task_identifier": "facets_drivers.simulation.RunLammps",
    },
]

links = [
    {
        "source": "get_orientation",
        "target": "nanoresculpted",
        "data_mapping": [{"source_output": "nsparam_filepath", "target_input": "nsparam_filepath"}],
    },
    {
        "source": "nanoresculpted",
        "target": "lammps",
        "data_mapping": [{"source_output": "pos_filepath", "target_input": "pos_filepath"}],
    },
]

# `links` non-mandatory if single computation
workflow = {"graph": {"id": "hello_world"}, "nodes": nodes, "links": links}

# Use editor, should be better
inputs = [
    {"id": "get_orientation", "name": "scratch_dirpath", "value": simulation_dump_directory},
    {"id": "get_orientation", "name": "vtp_facets_filepath", "value": f"{theory_dump_directory}/exp_1.vtp"},
    
    {"id": "nanoresculpted", "name": "scratch_dirpath", "value": simulation_dump_directory},
    {"id": "nanoresculpted", "name": "obj_filepath", "value": f"{theory_dump_directory}/exp.obj"},
    {"id": "nanoresculpted", "name": "num_procs", "value": f"{num_cores}"},

    {"id": "lammps", "name": "scratch_dirpath", "value": simulation_dump_directory},
    {"id": "lammps", "name": "num_procs", "value": f"{num_cores}"},
]

In [4]:
result = execute_graph(workflow, inputs=inputs)

If this is slow (few seconds), you do not have a GPU.
Detected XML VTK file format (.vtp).
Found 'FacetIds' in data source: vtkCellData
Found 'Normals' in data source: vtkCellData
Found 31 facets in exp_1.vtp.

--- Generating Theoretical Planes ---
Including families: [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
Generated 26 unique planes from families.
Excluding 2 specified planes: [(1, 1, 1), (-1, -1, -1)]
Removed 2 planes. 24 remain.

--- Starting Coarse Search (Targeted Wobble) ---
Searching with wobble angle up to 5.0° and tolerance 5.0°.
Searching 8712 total orientations...
  ... searched 5000 / 8712 orientations
Coarse search complete. Found a max of 18 matched facets.

--- Analysis Complete (Coarse Search) ---
Found a coarse rotation with 18 matched facets during the search.

Coarse Orientation Matrix (from Wobble search):
[[ 0.1694  0.5876 -0.7912]
 [-0.7392  0.6068  0.2924]
 [ 0.6519  0.5353  0.5371]]

--- Starting High-Precision Refinement ---
Refining using only inliers within 5.0°.
I